## Importing libraries

In [ ]:
# Data manipulation modules--
import pandas as pd
import numpy as np

# Silencing the warnings
import warnings
warnings.filterwarnings("ignore")

# Visualization packages--
%matplotlib inline
import seaborn as sns
import matplotlib.pyplot as plt

import missingno as msno 


# Imputation - Missing values

Missing values are one of the most common problems one encounters during data preparation. They affect the performance of the machine learning models to a greater extent.

For the following exercises, the volcano dataset has been used.

In [ ]:
volcano = pd.read_csv("volcano_data.csv")
volcano.head()

In [ ]:
pd.read_csv("Data Dictionary.csv",index_col=0)

In [ ]:
# Sum of nulls in the dataset
print("There are {0} missing values in the dataset.".format(volcano.isna().sum().sum()))

In [ ]:
# Visualize missing values as a matrix 
msno.matrix(volcano) 

The most simple solution to the missing values is to drop the rows or the entire column. There is no set threshold for dropping rows/columns and no method to mathematically determine the optimum threshold. In this case 70% is used as a threshold, i.e. rows/columns which have missing values more than this threshold are removed.

In [ ]:
# Setting the threshold to 70%
threshold = 0.7
data = volcano
#Dropping columns with missing value rate higher than threshold
data = data[data.columns[data.isnull().mean() < threshold]]

#Dropping rows with missing value rate higher than threshold
data = data.loc[data.isnull().mean(axis=1) < threshold]

# Before treating the missing values
print("Before dropping the records with missing values - {0}.".format(volcano.shape))
print("After dropping the records - {0}.".format(data.shape))

### Numerical Imputation
**Imputation is preferable compared to dropping rows/columns as it preserves data size**. In this case, it is important to select the imputation value accurately. It is generally desirable to have a default value which replaces missing values. This is especially easy for binary variables, for example, if a column has 1 and NA, then it is likely that the NA rows correspond to 0. However, the same cannot be necessarily performed for variables on a ratio scale. For example, the missing values of the variable “number of customer visits in the previous month”, should be replaced with 0 only if one were sure that no customers could have visited that outlet in the last month

One of the reasons for the presence of missing values is when tables with different sizes are joined together, in which case imputing with 0 is reasonable.

Except when a default imputation value exists, the best imputation technique is using the medians of respective column. The median medians is preferred as it is the least sensitive to outliers compared to other measures of central tendency, and thus is the most representative of the data.

In [ ]:
## Filtering for numerical columns
numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
numericaldf = volcano.select_dtypes(include=numerics)

print("Here's is the list of numerical variables - {0}".format(numericaldf.columns))

In [ ]:
print("Missing values before imputation -\n{0}".format(numericaldf.isna().sum()))

In [ ]:
# Visualize missing values as a matrix 
msno.matrix(numericaldf) 


In [ ]:
#Filling missing values with medians of the columns
data = numericaldf
data = data.fillna(data.median())

print("Filling missing values with medians of the columns\n")

print("Missing values after imputating median values -\n{0}".format(data.isna().sum()))

In [ ]:
# Visualize missing values as a matrix 
msno.matrix(data) 

### Categorical Imputation

Replacing the missing values with the mode (maximum occurred value) of a column is desirable when handling categorical columns. But if the values in the column are distributed uniformly and there no dominant value, imputing a category like “Others” might be more sensible, because in such cases, the imputation is likely to converge to a random selection.

In [ ]:
categoricaldf = volcano.select_dtypes(exclude=numerics)

print("Here's is the list of categorical variables - \n{0}".format(categoricaldf.columns))

In [ ]:
print("Missing values before imputation -\n{0}".format(categoricaldf.isna().sum()))

In [ ]:
# Visualize missing values as a matrix 
msno.matrix(categoricaldf) 

The column Agent column has 40 null values. Here the mode (maximum occurring) value is imputed in place of all missing values.

In [ ]:
#Max fill function for categorical columns
data = categoricaldf
data['Agent'].fillna(data['Agent'].value_counts().idxmax(), inplace=True)

The missing values in the columns **TSU** and **EQ** are where the volcano didn't cause a Tsunami or an earthquake. So replacing them with mode will cause errors in the data. Hence replacing the values with **'No TSU'** and **'No EQ'**.

In [ ]:
data['TSU'].fillna("No TSU", inplace=True)
data['EQ'].fillna("No EQ", inplace=True)

In [ ]:
print("Missing values after imputation -\n{0}".format(data.isna().sum()))

In [ ]:
# Visualize missing values as a matrix 
msno.matrix(data) 


As seen above, TSU and EQ have no missing values

# Handling Outliers

An outlier is a data point that differs significantly from other observations. An outlier occurs due to variability in measurement or due to an experimental error, with the latter mostly being excluded from the data. An outlier can cause serious problems in statistical analyses. There are two different ways of handling outliers – using standard deviation, and percentiles. Here we are using the iris flowers dataset understanding the outliers.

In [ ]:
iris = sns.load_dataset("iris")

In [ ]:
plt.figure(figsize=(10,6))
ax = sns.boxplot(data=iris, orient="h", palette="Set1")

### Outlier Detection with Standard Deviation
If a value is located at a distance greater than n standard deviations, then it is considered as an outlier. How is n determined? While there is no definite mathematical solution to determine n, a value between 2 and 4 usually provides the best results in practice

In [ ]:
#Dropping the outlier rows with standard deviation
factor = 3
data = sns.load_dataset("iris")
upper_lim = data['sepal_width'].mean () + data['sepal_width'].std () * factor
lower_lim = data['sepal_width'].mean () - data['sepal_width'].std () * factor

data = data[(data['sepal_width'] < upper_lim) & (data['sepal_width'] > lower_lim)]

In [ ]:
plt.figure(figsize=(10,6))
ax = sns.boxplot(data = data, orient="h", palette="Set1")

In addition, z-score can be used instead of the formula above. Z-score (or standard score) standardizes the distance between a value and the mean using the standard deviation.

### Outlier Detection with Percentiles
Another mathematical method to detect outliers is to use percentiles. The Inter Quartile Range (IQR) rule is useful in detecting the presence of outliers. In general, any number which lies above or below 1.5IQR is considered as an outlier.

In [ ]:
#Dropping the outlier rows with Percentiles
data = sns.load_dataset("iris")
q3 = data['sepal_width'].quantile(.75)
q1 = data['sepal_width'].quantile(.25)
IQR = q3 - q1

upper_lim = q3 + (1.5 * IQR)
lower_lim = q1 - (1.5 * IQR)


data = data[(data['sepal_width'] < upper_lim) & (data['sepal_width'] > lower_lim)]

In [ ]:
plt.figure(figsize=(10,6))
ax = sns.boxplot(data = data, orient="h", palette="Set1")

### An Outlier Dilemma: Drop or Cap
Another option for handling outliers is to cap them instead of dropping so that the data size remains same and hence the model can be trained well. On the other hand, capping can affect the distribution of the data, thus it is better not to treat.

In [ ]:
#Capping the outlier rows with Percentiles
data = iris
q3 = data['sepal_width'].quantile(.75)
q1 = data['sepal_width'].quantile(.25)
IQR = q3 - q1

upper_lim = q3 + (1.5 * IQR)
lower_lim = q1 - (1.5 * IQR)

data.loc[(data['sepal_width'] > upper_lim),'sepal_width'] = upper_lim
data.loc[(data['sepal_width'] < lower_lim),'sepal_width']= lower_lim

In [ ]:
plt.figure(figsize=(10,6))
ax = sns.boxplot(data = data, orient="h", palette="Set1")